In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
from src.driver.service import DriverService
from src.state_manager.state import StateManager
from src.actions.browser_actions import BrowserActions
from src.agent_interface.planing_agent import PlaningAgent
from src.actions.browser_actions import ActionResult
from src.state_manager.utils import save_formatted_html

import os
from datetime import datetime

In [113]:
driver = DriverService().get_driver()
state_manager = StateManager(driver)
actions = BrowserActions(driver)

In [114]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
run_folder = f'temp/run_{timestamp}'
if not os.path.exists(run_folder):
    os.makedirs(run_folder)

In [115]:
task = 'Go to kayak.com and find a flight from Zürich to (ask the user for the destination) on 2025-04-25 with return on 2025-06-05 for 2 people.'

In [116]:
print(actions.get_default_actions())

{'ask_user': 'text: string', 'send_user_text': 'text: string', 'search_google': 'text: string', 'go_to_url': 'url: string', 'done': '', 'go_back': '', 'click': 'id: int', 'input': 'id: int, text: string', 'nothing': ''}


In [117]:
agent = PlaningAgent(default_actions=actions.get_default_actions(), model="gpt-4o")

In [118]:
agent.add_user_prompt(f'Your task is: {task}', after_system=True)

In [119]:
url_history = []
output = ActionResult()

In [120]:
i = 0

In [121]:
previous_state = None

---

In [345]:
current_state = state_manager.get_current_state(only_top=True)

In [346]:
state_manager.add_change_info(current_state=current_state, previous_state=previous_state)
state_manager.add_parent_info(current_state=current_state)

In [347]:
save_formatted_html(driver.page_source, f'{run_folder}/html_{i}.html')

In [348]:
url_history.append(driver.current_url)

In [349]:
elem_text = "\n".join([e.get_text() for e in current_state.interactable_elements])
text = f'Elements:\n{elem_text}\nUrl history: {url_history}'

In [350]:
print(text)

Elements:
      6:<div aria-label="Open main navigation"></div>
      7:<a href="/" aria-label="Go to the kayak homepage"></a>
     35:<div aria-label=" Round-trip">Round-trip</div>
     36:	<span>Round-trip</span>
     38:<div aria-label=" ISB, ZRH">From? | ISB, ZRH</div>
     41:	<span>ISB, ZRH</span>
     43:<div aria-label=" Denpasar, Bali, Indonesia">To? | Denpasar, Bali, Indonesia</div>
     46:	<span>Denpasar, Bali, Indonesia</span>
     48:<div aria-label=" Fri 4/25">Add dates | Fri 4/25</div>
     51:	<span>Fri 4/25</span>
     53:<div aria-label=" Thu 6/5">Add dates | Thu 6/5</div>
     56:	<span>Thu 6/5</span>
     58:<div aria-label=" 2 travelers, Economy">2 travelers, Economy</div>
     59:	<span>2 travelers, Economy</span>
     61:<button aria-disabled="false" type="submit" aria-label="Search"></button>
     62:<div aria-label="Open Trips drawer"></div>
     63:<div>Sign in</div>
     64:	<span>Sign in</span>
     66:<a href="/flights" aria-label="Search for flights " ari

In [351]:
if output.user_input:
    agent.add_user_prompt(output.user_input)
if output.error:
    text += f', Previous action error: {output.error}'
action = agent.chat(text, store_conversation=f'{run_folder}/conversation_{i}.txt')

Total cost: $0.0187 for 7364 tokens


In [352]:
action

Action(valuation_previous_goal='Successfully informed the user of the available flight options with their details.', goal='Complete the task', action='done', params=ActionParams(url=None, id=None, text=None))

In [353]:
output = actions.execute_action(action, current_state.selector_map)

{
    "valuation_previous_goal": "Successfully informed the user of the available flight options with their details.",
    "goal": "Complete the task",
    "action": "done",
    "params": {
        "url": null,
        "id": null,
        "text": null
    }
}


In [354]:
if output.done:
    print('Task completed')
    driver.quit()

Task completed


In [355]:
i += 1

In [356]:
previous_state = current_state

---

In [110]:
driver.quit()